分段式 Quasicrystal 圖片產生器
===
本程式修改自 [QuasicrystalGifs](https://github.com/makeyourownmaker/QuasicrystalGifs)。雖然程式碼有些差異，但產生 quasicrystal 圖片、動態圖片和著色的原理並沒有改變。

###程式的改變在於：
  1. 讓大部分未指定的參數隨機產生，每次執行都能產生不同圖片。
  2. 輸出上次執行的參數，可以把喜歡圖片的參數保留，或是保留部分，用來下次產生圖片。
  3. 在 Jupyter Notebook 分段執行，可以從灰階、彩圖到動畫分段觀察和生成。
  4. 加了一個漸進式參數變化的動畫。

###工作流說明：
  1. **初始化函式庫**：定義所有幾何計算與參數管理類別。
  2. **計算 quasicrystal 相位資料**：隨機決定波數、頻率等幾何基礎，先以灰階預覽。
  3. **產生圖片**：加入 Colormap 與 LightSource 光照效果。
  4. **產生動畫**：產生基本的波動動畫。
  5. **漸進式參數動畫**：產生波數或縮放隨時間變化的進階動畫。


初始化函式庫.

In [ ]:
%matplotlib inline
import numpy as np
from math import pi
from math import sqrt
from math import ceil
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from google.colab import drive
from matplotlib.animation import PillowWriter
import random
import matplotlib
import json
import matplotlib.image as mpimg

# 輔助函數：隨機回傳 True 或 False
def RandBool():
  if random.randint(0,1) == 0:
    return True
  else:
    return False

# 產生標準的線性波陣列 (Standard Line Array)
# 透過在不同角度 (allwaves) 上產生的平面波疊加
def GetStdLineArray(gridX,gridY,waves,freq,offset=0,custom_waves=None):
  # 計算對應的波方向角度
  allwaves=(np.arange(0,pi,pi/waves)+offset/waves) if custom_waves is None else custom_waves
  # 利用投影公式 x*cos(theta) + y*sin(theta) 計算相位
  xcos=gridX*np.cos(allwaves)[:,np.newaxis,np.newaxis]
  ysin=gridY*np.sin(allwaves)[:,np.newaxis,np.newaxis]
  ret=(xcos+ysin)*freq;
  return ret

# 產生極座標波陣列 (Polar Line Array)
# 會產生旋轉對稱或圓環狀的干涉圖案
def GetPolarLineArray(gridX,gridY,waves,freq,offset=0,custom_waves=None,sq=True,log=True):
  allwaves=(np.arange(0,pi,pi/waves)+offset/waves) if custom_waves is None else custom_waves
  theta=np.arctan2(gridX, gridY)    # 計算角度
  r=gridX*gridX+gridY*gridY       # 計算距離平方
  if sq: r=np.sqrt(r)         # 是否取平方根 (線性距離)
  if log: r=np.log(r)         # 是否取對數 (產生向中心縮小的效果)
  r[np.isinf(r) is True] = 0     # 處理對數產生的無限大數值
  tcos=theta*np.cos(allwaves)[:,np.newaxis,np.newaxis]
  rsin=r*np.sin(allwaves)[:, np.newaxis, np.newaxis]
  ret=(tcos+rsin)*freq;
  return ret

# 閉包函數：用來快速生成特定設定的極座標計算函數
def GeneratePolarFunc(sq,log):
  def retFunc(gridX,gridY,waves,freq,offset=0,custom_waves=None):
    return GetPolarLineArray(gridX,gridY,waves,freq,offset,custom_waves,sq,log)
  return retFunc

# 核心函數：產生準晶體的相位矩陣
# width/height: 圖片解析度, scale: 座標縮放, freq: 頻率(影響條紋密度)
def GenerateQuasiCrystalPhase(waves,freq,width,height,scale=1,offset=0,jump_zero=False,custom_waves=None,func=GetStdLineArray):
  r=pi*scale
  # 建立網格座標系統
  if jump_zero:
    dx = np.arange(-r+r/width, r+r/width, 2*r/width,dtype=np.float64)
    dy = np.arange(-r+r/height, r+r/height, 2*r/height,dtype=np.float64)
  else:
    dx = np.arange(-r, r, 2*r/width,dtype=np.float64)
    dy = np.arange(-r, r, 2*r/height,dtype=np.float64)
  xv, yv = np.meshgrid(dx, dy)
  # 根據指定的函數 (標準或極座標) 產生相位
  return func(xv,yv,waves,freq,offset,custom_waves)

#   影像生成器：處理光照與著色 (Shading)
#   blend_mode: 混合模式 ('hsv', 'overlay', 'soft'), azdeg/altdeg: 光源方位角與高度角
#   顏色來自 plt 配色給的選擇
#  cmap=['Accent', 'Blues', 'BrBG', 'BuGn',    'BuPu', 'CMRmap', 'Dark2', 'GnBu', 'Greens', 'Greys', 'OrRd', 'Oranges', 'PRGn', 'Paired',
#       'Pastel1', 'Pastel2', 'PiYG', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy', 'RdPu', 'RdYlBu', 'RdYlGn', 'Reds',
#       'Set1', 'Set2', 'Set3', 'Spectral', 'Wistia', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd', 'afmhot', 'autumn', 'binary', 'bone', 'brg',
#       'bwr', 'cividis', 'cool', 'coolwarm', 'copper', 'cubehelix', 'flag', 'gist_earth', 'gist_gray', 'gist_heat', 'gist_ncar',
#       'gist_rainbow', 'gist_stern', 'gist_yarg', 'gnuplot', 'gnuplot2', 'gray', 'hot', 'hsv', 'inferno', 'jet', 'magma', 'nipy_spectral',
#       'ocean', 'pink', 'plasma', 'prism', 'rainbow', 'seismic', 'spring', 'summer', 'tab10', 'tab20', 'tab20b', 'tab20c', 'terrain',
#       'turbo', 'twilight', 'twilight_shifted', 'viridis', 'winter']
# or add _r, like 'gray_r'

def GenerateImageProducer(blend_mode,azdeg,altdeg,vexag):
  AllBlendMode=['hsv','overlay','soft']
  func=None
  if blend_mode not in AllBlendMode:
    def func(image,cmap):
      return image    # 若無混合模式則直接回傳原圖
  else:
    ls = LightSource(azdeg=azdeg, altdeg=altdeg)
    def func(image,cmap):
      # 使用 Matplotlib 的 LightSource 產生具有立體感的陰影效果
      return ls.shade(image,cmap=plt.get_cmap(cmap),blend_mode=blend_mode,vert_exag=vexag)
  return func

# 核心繪製函數：將相位數據轉換為 RGB 影像
def GenerateQuasiCrystalImage(data,image,cmap,imageFunc,calcFunc=None):
  if calcFunc is None: calcFunc = lambda data : np.cos(data)
  if imageFunc is None: imageFunc = GenerateImageProducer(None,0,0,0)
  # 將所有波疊加並取餘弦值 (準晶體公式核心)
  image[:] = np.sum(calcFunc(data), axis=0)
  rgb = imageFunc(image,cmap)
  im = plt.imshow(rgb,cmap=cmap)
  return im

# 顯示靜態準晶體圖片
def ShowQuasiCrystalImage(data,image,cmap,blend_mode='None',azdeg=250,altdeg=50,vexag=1,calcFunc=None):
  imageFunc = GenerateImageProducer(blend_mode,azdeg,altdeg,vexag)
  return GenerateQuasiCrystalImage(data,image,cmap,imageFunc)

# 產生準晶體動畫：改變相位
# frames: 總幀數, delay: 幀間隔時間
def ShowQuasiCrystalAnimate(data,image,figure,frames,delay,cmap,blend_mode='None',azdeg=250,altdeg=50,vexag=1,calcFunc=None,path=None):
  if calcFunc is None: calcFunc = lambda data : np.cos(data)
  phases = np.arange(0,2*pi,2*pi/frames)      # 一個完整的 2pi 週期動畫
  imageFunc = GenerateImageProducer(blend_mode,azdeg,altdeg,vexag)

  def animate_func(i):
    adata=data+phases[i]              # 隨時間改變相位
    im = GenerateQuasiCrystalImage(adata,image,cmap,imageFunc,calcFunc)
    return [im]

  ani = animation.FuncAnimation(figure,animate_func,frames=frames,interval=delay)
  if path is not None:
    ani.save(path,writer='pillow')
  return ani;

# 進階動畫函數：可動態改變縮放、波數量、頻率等參數
def CalcQuasiCrystalAnimate(width,height,scale_array,waves_array,freq_array,offset_array,phase_array,image,figure,frames,delay,cmap,func,jump_zero=False,blend_mode='None',azdeg=250,altdeg=50,vexag=1,calcFunc=None,path=None):
  if calcFunc is None: calcFunc = lambda data : np.cos(data)
  imageFunc = GenerateImageProducer(blend_mode,azdeg,altdeg,vexag)

  # 取得各參數數組長度，用於循環取值 (Modulo)
  ls=len(scale_array)
  lw=len(waves_array)
  lf=len(freq_array)
  lo=len(offset_array)
  lp=len(phase_array)

  def animate_func(i):
    # 每一幀都重新計算相位網格
    data=GenerateQuasiCrystalPhase(waves_array[i%lw],freq_array[i%lf],width,height,scale_array[i%ls],offset_array[i%lo],jump_zero,custom_waves=None,func=func)
    adata=data+phase_array[i%lp]
    im = GenerateQuasiCrystalImage(adata,image,cmap,imageFunc,calcFunc)
    return [im]

  ani = animation.FuncAnimation(figure,animate_func,frames=frames,interval=delay)
  if path is not None:
    ani.save(path,writer='pillow')
  return ani

# 管理參數的類別。
# 功能：
# 1. 紀錄部分參數（透過 JSON 格式）。
# 2. 如果要取得的參數沒有預設值，會根據資料型態自動產生亂數回傳，方便隨機藝術創作。
class ParamSaver:
  def __init__(self):
    # 存放參數的字典
    self.params={}

  # 將目前的參數轉換為格式化的 JSON 字串
  def output(self):
    return json.dumps(self.params,indent=4)

  # 從 JSON 字串載入參數
  def load(self, json_string):
    self.params=json.loads(json_string)

  # 設定參數，若 value 為空則不動作
  def set(self, key, value):
    if value is not None:
      self.params[key]=value

  # 基礎取得函式：若鍵值不存在則回傳 None
  def get(self, key):
    try:
      return self.params[key]
    except:
      return None

  # 取得布林值：若無設定則回傳隨機 True/False
  def getBool(self, key):
    v=self.get(key)
    if v is None:
      return RandBool()
    else:
      return v

  # 取得整數：若無設定，且有提供 max 則回傳範圍亂數，否則回傳 min 預設值
  def getInt(self, key, min, max=None):
    v=self.get(key)
    if v is None:
      if max==None:
        return min
      else:
        return random.randint(min,max)
    else:
      return int(v)

  # 取得字串：若無設定則回傳 default 預設值
  def getString(self,key,default):
    v=self.get(key)
    if v is None:
      return default
    else:
      return v

  # 取得浮點數：若無設定，則在 [min, max) 範圍內產生亂數
  def getFloat(self, key, min, max=None):
    v=self.get(key)
    if v is None:
      if max==None:
        return min
      else:
        return random.random() * (max - min) + min
    else:
      return float(v)

  # 從列表 (choices) 中挑選：若無設定且 toChoice 為真，則隨機選一個
  def getChoice(self, key, choices, toChoice=True):
    v=self.get(key)
    if v is None and toChoice:
      return random.choice(choices)
    else:
      return v

  # 取得數值陣列（用於動畫序列）：
	# 可以產生等差級數或週期性（鏡像對稱）的陣列
  def getNumericArray(self, key, min, maxStep, length, periodic=False):
    v=self.get(key)
    if v is list:
      return v;
    # 隨機決定是否不產生陣列，僅回傳包含單一值的清單
    if maxStep==0 or length <= 0 or RandBool():
      return [min]
    # 決定遞增/遞減步長
    if type(maxStep)==int:
      step=random.randint(1,maxStep) if maxStep > 0 else random.randint(-maxStep,0)
    else:
      step=maxStep
    if periodic==False:
      # 產生一般的等差數列
      return [min+i*step for i in range(0,length)]
    # 產生鏡像對稱的數列（例如 1,2,3,2,1），使動畫循環銜接順暢
    if length%2==0:
      half_ret = [min+i*step for i in range(0,int(length/2)+1)]
      return half_ret+half_ret[::-1][1:-1]
    else:
      helf_ret = [min+i*step for i in range(0,int(length/2)+1)]
      return helf_ret+helf_ret[::-1][:-1]

  # 判斷參數是否為陣列型態
  def hasNumericArray(self, key):
    v=self.get(key)
    return v is list

  def generateInfoString(self):
    w=self.getInt("waves",0,)
    f=self.getInt("freq",0,)
    m=self.getString("calcFunc","None")
    cmap=self.getString("colormap","None")
    if m=="polar" or m=="logpolar":
      s=self.getInt("scale",0)
      return "waves_%d_freq_%d_%s%d_colormap_%s" % (w,f,m,s,cmap)
    else:
      return "waves_%d_freq_%d_colormap_%s" % (w,f,cmap)

  # 生成描述字串：用於存檔時自動命名，包含波數、頻率、計算方式與色表資訊
  def save(self, path):
    v=self.output()
    try:
      with open(path, 'w', encoding='utf-8') as f:
        f.write(v)
      print(f"Success to write string to {path}")
    except IOError as e:
      print(f"Write IO error: {e}")

  # 將目前的參數字典存入檔案 (JSON)
  def loadFile(self, path):
    try:
      with open(path, 'r', encoding='utf-8') as f:
        self.load(f.read())
      print(f"Success to read string from {path}")
    except IOError as e:
      print(f"Read IO error: {e}")

##初始化參數，不能略過。*json_string* 可以填入參數，空物件代表全部使用程式預設值或亂數。

In [ ]:
json_string='{}'
input=ParamSaver()
input.load(json_string)
output=ParamSaver()

###計算 quasicrystal 相位資料，以灰階圖片顯示。這個階段用到以下參數：

<dl>

<dt>width & heigh</dt><dd>畫布尺寸。定義輸出的影像寬度與高度（像素）。預設 512。</dd>

<dt>scale</dt><dd>縮放比例。決定座標平面的範圍。數值越大，看到的圖案範圍越廣（圖案會顯得越小、越密集）。預設最小值是 1。</dd>



<dt>wave</dt><dd>平面波數量。準晶體是由多個方向的平面波疊加而成。 : 例如：waves=3 會產生六角對稱，waves=5 會產生經典的五角準晶對稱。預設最小值是 4。</dd>



<dt>freq</dt><dd>頻率。影響波紋的密集程度。高頻率會產生更細碎、更複雜的干涉紋理。預設最小值是 2。</dd>


<dt>offset</dt><dd>初始偏移量。改變波的起始角度，這會讓整個對稱圖案產生旋轉或結構性的位移。沒有指定的話會從 0 到 $\pi$ 隨機選擇一個值。</dd>

<dt>jump_zero</dt><dd>跳過原點。布林值（True/False）。
:   若為 True，計算時會稍微偏移中心點，避免在極座標轉換（如 logpolar）時因為 $r=0$ 導致數值無限大（Singularity）。</dd>

<dt>calcFuncName</dt><dd>座標轉換函式。決定圖案的分佈邏輯：<b>default</b>：標準平面干涉。 <b>polar / sqrtpolar</b>：極座標轉換，產生放射狀或螺旋狀圖案。<b> logpolar / sqrtlogpolar</b>：對數極座標轉換，產生具分形（Fractal）感的深度收縮效果。</dd>
</dl>

###以下參數不是直接決定圖形，而是決定當不指定參數時，亂數產生亂數的最大值：
<dl>
<dt>maxScale</dt><dd>亂數產生的最大縮放比例(scale)，預設 10。</dd>
<dt>maxWave</dt><dd>亂數產生的最大平面波數量(wave)，預設 128.</dd>
<dt>maxFreq</dt><dd>亂數產生的最大頻率(freq)，預設 64.</dd>
</dl>

In [ ]:
width = input.getInt("width",512)
height = input.getInt("height",512)
scale=input.getInt("scale",1,input.getInt("maxScale",10))
waves = input.getInt("waves",4,input.getInt("maxWaves",128))
freq = input.getInt("freq",2,input.getInt("maxFreq",64))
offset = input.getFloat("offset",0,pi if RandBool() else None)
jump_zero = input.getBool("jump_zero")
AllFunc=['default','polar','sqrtpolar','logpolar','sqrtlogpolar']
calcFuncName=input.getChoice("calcFunc",AllFunc)

if calcFuncName=='polar':
  func = GeneratePolarFunc(False,False)
elif calcFuncName=='logpolar':
  func = GeneratePolarFunc(False,True)
elif calcFuncName=='sqrtpolar':
  func = GeneratePolarFunc(True,False)
elif calcFuncName=='sqrtlogpolar':
  func = GeneratePolarFunc(True,True)
else:
  func = GetStdLineArray

output.set("width",width)
output.set("height",height)
output.set("scale",scale)
output.set("waves",waves)
output.set("freq",freq)
output.set("offset",offset)
output.set("jump_zero",jump_zero)
output.set("calcFunc",calcFuncName)

inner=GenerateQuasiCrystalPhase(waves,freq,width,height,scale,offset,jump_zero,func=func)

image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ShowQuasiCrystalImage(inner,image,'gray')

輸出上一個 cell 產生圖片的參數。

In [ ]:
print(output.output())

###產生圖片，到這裡再來決定圖片的配色，加入隨機的色彩表與 LightSource 陰影模式。這裡可以重複執行到出現自己喜歡的配色。這個階段用到這些參數：
<dl>

<dt>colormap</dt><dd>色表。使用 Matplotlib 的色彩映射決定數據深淺對應的顏色。</dd>

<dt>blend_mode</dt><dd>陰影混合模式。利用 ls.shade 產生的立體視覺效果：<b>None</b>：純 2D 顏色呈現。<b>hsv</b>：顏色較為鮮豔飽和。<b>overlay</b>：高對比，強化亮部與暗部。<b>soft</b>：較柔和的立體感，類似噴砂表面。</dd>

<dt>azdeg</dt><dd>Azimuth，光源方位角。光源在水平面上的角度（0-360°），0° 代表從正北方射入。</dd>

<dt>altdeg</dt><dd>Altitude，光源高度角。光源相對於地平線的高度（0-90°），90° 為直射。</dd>

<dt>vexag</dt><dd>Vertical Exaggeration，垂直誇張係數。數值越高，圖案的「坡度」越陡峭，陰影越深，立體感越強烈。</dd>

</dl>

這裡也寫下目前 Matplotlib 內的配色名稱

['Accent', 'Blues', 'BrBG', 'BuGn',    'BuPu', 'CMRmap', 'Dark2', 'GnBu', 'Greens', 'Greys', 'OrRd', 'Oranges', 'PRGn', 'Paired',
'Pastel1', 'Pastel2', 'PiYG', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy', 'RdPu', 'RdYlBu', 'RdYlGn', 'Reds',
'Set1', 'Set2', 'Set3', 'Spectral', 'Wistia', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd', 'afmhot', 'autumn', 'binary', 'bone', 'brg',
'bwr', 'cividis', 'cool', 'coolwarm', 'copper', 'cubehelix', 'flag', 'gist_earth', 'gist_gray', 'gist_heat', 'gist_ncar',
'gist_rainbow', 'gist_stern', 'gist_yarg', 'gnuplot', 'gnuplot2', 'gray', 'hot', 'hsv', 'inferno', 'jet', 'magma', 'nipy_spectral',
'ocean', 'pink', 'plasma', 'prism', 'rainbow', 'seismic', 'spring', 'summer', 'tab10', 'tab20', 'tab20b', 'tab20c', 'terrain',
'turbo', 'twilight', 'twilight_shifted', 'viridis', 'winter']

這些名稱還能加上 '_r' 代表同一種色表的反向順序。

In [ ]:
cmap = input.getChoice("colormap",plt.colormaps())
AllBlendModes = ['hsv','overlay','soft']
blend_mode=input.getChoice("blend_mode",AllBlendModes,RandBool())

if blend_mode in AllBlendModes:
  azdeg=input.getInt("azdeg",0,360)
  altdeg=input.getInt("altdeg",0,90)
  vexag=input.getInt("vexag",1,10)
else:
  azdeg=None
  altdeg=None
  vexag=None

output.set("colormap",cmap)
if blend_mode in AllBlendModes:
  output.set("blend_mode",blend_mode)
  if azdeg is not None:
    output.set("azdeg",azdeg)
  if altdeg is not None:
    output.set("altdeg",altdeg)
  if vexag is not None:
    output.set("vexag",vexag)
else:
  output.set("blend_mode","None")

image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ShowQuasiCrystalImage(inner,image,cmap,blend_mode,azdeg,altdeg,vexag)

輸出上一個 cell 產生圖片的參數。比上一次輸出時多了視覺渲染參數。

In [ ]:
print(output.output())

###產生動畫。這裡用到以下參數：
<dl>

<dt>frames</dt><dd>總幀數。動畫循環一次所包含的圖片張數。</dd>

<dt>delay</dt><dd>幀間延遲。每張圖之間的毫秒數，決定動畫播放的速度。</dd>

</dl>

##注意，這個步驟比較費時。

In [ ]:
frames=input.getInt("frames",24)
delay=input.getInt("delay",100)
matplotlib.rcParams['animation.embed_limit']=50
image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

ani=ShowQuasiCrystalAnimate(inner,image,fig,frames,delay,cmap,blend_mode,azdeg,altdeg,vexag)
HTML(ani.to_jshtml())

儲存成 gif 檔案，如果在 colab，你可以從左邊尋找檔案列表來下載圖片。
##注意，這個步驟比較費時。

In [ ]:
image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ani=ShowQuasiCrystalAnimate(inner,image,fig,frames,delay,cmap,blend_mode,azdeg,altdeg,vexag)
name=output.generateInfoString()+".gif"

ani.save(name, writer='pillow')

另一種圖片，一部分參數會漸進式變化，這個階段用到這些參數：
<dl>

<dt>frames</dt><dd>總幀數。動畫循環一次所包含的圖片張數。</dd>

<dt>delay</dt><dd>幀間延遲。每張圖之間的毫秒數，決定動畫播放的速度。</dd>

<dt>waves_array / freq_array / scale_array</dt><dd>動態參數數列。這是在「進階動畫」中使用的數列。 : 當這些參數隨時間改變時，圖案會產生像心跳般縮放或結構重組（例如波數從 5 變到 12）的動態效果。</dd>

<dt>periodic</dt><dd>週期性循環。用於 getNumericArray。 : 若為 True，產生的數列會是對稱的（例如 1, 2, 3, 2, 1），這能讓動畫在結束時完美銜接回開頭，形成無縫循環。</dd>
</dl>

##注意，這個步驟比較費時。

In [ ]:
frames=input.getInt("frames",20,30)
delay=input.getInt("delay",100)
image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

offset_array=input.getNumericArray("offset_array",offset,offset,frames,RandBool())
phase_array=input.getNumericArray("phase_array",0,2*pi/frames,frames)
waves_array=input.getNumericArray("waves_array",waves,input.getInt("max_waves_step",4),frames,RandBool())
freq_array=input.getNumericArray("freq_array",freq,input.getInt("max_freq_step",6),frames,RandBool())
scale_array=input.getNumericArray("scale_array",scale,input.getInt("max_scale_step",ceil(scale*0.1)),frames,RandBool())

testTrival=not(input.hasNumericArray("waves_array") and input.hasNumericArray("freq_array") and input.hasNumericArray("scale_array"))
while testTrival:
  if len(waves_array)>1 or len(freq_array)>1 or len(scale_array)>1:
    break
  waves_array=input.getNumericArray("waves_array",waves,input.getInt("max_waves_step",4),frames,RandBool())
  freq_array=input.getNumericArray("freq_array",freq,input.getInt("max_freq_step",6),frames,RandBool())
  scale_array=input.getNumericArray("scale_array",scale,input.getInt("max_scale_step",ceil(scale*0.1)),frames,RandBool())


output.set("frames",frames)
output.set("delay",delay)
output.set("waves_array",waves_array)
output.set("freq_array",freq_array)
output.set("offset_array",offset_array)
output.set("phase_array",phase_array)
output.set("scale_array",scale_array)


ani=CalcQuasiCrystalAnimate(width,height,scale_array,waves_array,freq_array,offset_array,phase_array,image,fig,frames,100,cmap,func,jump_zero,blend_mode,azdeg,altdeg,vexag)
HTML(ani.to_jshtml())

輸出上一個 cell 產生圖片的參數。比上一次輸出時多了漸進式圖片需要的參數。

In [ ]:
output.save(output.generateInfoString()+'.json')
print(output.output())

儲存成 gif 檔案，如果在 colab，你可以從左邊尋找檔案列表來下載圖片。

##注意，這個步驟比較費時。

In [ ]:
image = np.empty((height, width))
fig = plt.figure(figsize=(width/100, height/100))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ani=CalcQuasiCrystalAnimate(width,height,scale_array,waves_array,freq_array,offset_array,phase_array,image,fig,frames,100,cmap,func,jump_zero,blend_mode,azdeg,altdeg,vexag)
name=output.generateInfoString()+"_v.gif"

ani.save(name, writer='pillow')
